In [1]:
import cv2
import numpy as np
import itertools
from matplotlib import pyplot as plt

# Find convolution of 2 shape

In [2]:
def getMask(mask_path):
    # path to mask image
    mask = cv2.imread(mask_path,0)
    mask_position=set()
    for row, value in enumerate(mask):
        for column, element in enumerate(value):
            if element == 255:
                mask_position.add((column,row))
    return mask_position

def getBbox(bbox):
    x,y,x_plus_w,y_plus_h = bbox 
    vehicle_position = itertools.product(range(x,x_plus_w+1),range(y,y_plus_h+1))
    return set(vehicle_position)

def getProbability2Shape(vehicle, mask):
    match = len(vehicle)-len(vehicle.difference(mask))
    bbox  = len(vehicle)
    return (match*100)/bbox

'''
Just a mark for myself
'''
def get_output_layers(net):
    '''
    get all output layer names: with yolov3 is yolo_82, 94 and 106
    ''' 
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidences, x, y, x_plus_w, y_plus_h):
    '''
    draw a bounding box around object
    '''
    label = str(classes[class_id])

    color = COLORS[class_id]
    
#     print(x,y,x_plus_w,y_plus_h)
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def bounding_pic(net, image, scale, size,mask):
    '''
    net: the model reading by open-cv (maybe YoLo or SSD)
    image: image need to bouding box
    scale: image pixel multiply with this
    size: a tuple contain size of image
    '''
    Width = image.shape[1]
    Height = image.shape[0]
    # Resize picture to 416x416, because YOLO take in 416x416
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input is resized picture
    net.setInput(blob)
    # last layer of Yolo model
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.6
    # maybe our model will detect many bouding box for an object, this threshold help us take the box with equal 
    #            or higher propability
    nms_threshold = 0.4

    '''
    out is a 2D tensor like (number_of_objects, score_of_each_classes), with first five element in each row is special, 
    take e.g: out[0] = temp:
        + temp[0]: x_center of that object
        + temp[1]: y_center of that object
        + temp[2]: width of that object
        + temp[3]: height of that object
        + temp[4]: unknow value
        + from 5 to above is the score of that object to each classes => COCO have 80 class so each row contain 85 element,
            will be 15 with CIFAR,and 1005 with IMAGENET  
    '''
    for out in outs:
        for detection in out:
            scores = detection[5:]
            # get the highest score to determine its label
            class_id = np.argmax(scores)
            if class_id not in [0,1,2,3,7]:
                continue
            else:
                # score of that object, make sure more than 50% correct label
                confidence = scores[class_id]
                if confidence > 0.5:
                    # scale again with w and h
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    # remember it return x_center and y_center, not x,y, so we need to find x,y
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

    # detect bouding box around objects
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    # set the counting line
    lineThickness = 2
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = round(box[0])
        y = round(box[1])
        w = round(box[2])
        h = round(box[3])
        draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
        bbox = getBbox((x,y,x+w,y+h))
        conv = getProbability2Shape(bbox,mask)
        text1 = 'Convolution: ' + str(round(conv,2)) + '%'
        text2 = 'True' if conv > 80 else 'False'
        cv2.putText(image, text1, (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 5)
        cv2.putText(image, 'Fault: '+text2, (x, y+30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 5)
    return image

In [64]:
config = 'YOLOv3-416\\yolov3.cfg'
name = 'coco.names'
weight = 'YOLOv3-416\\yolov3.weights'
mask_path = './cross line/mask.jpg'

classes = None

with open(name, 'r') as f:
    # generate all classes of COCO, bicycle ind = 1, car ind = 2 and motorbike ind = 3
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Read the model
net = cv2.dnn.readNet(weight, config)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
# net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
# take shape of image in order to scale it to 416x416, first layer of Yolo CNN

scale = 0.00392

cap = cv2.VideoCapture('Z:\\Vehicle speed measuring\\video_speed measure\\front.avi')
'''
If output is not None
'''
fourcc = cv2.VideoWriter_fourcc(*'XVID')
outputVideo = cv2.VideoWriter('C:/Users/ADMINS/Desktop/areaResult.avi', fourcc, 20, (1920,1080))

#close

frameNo = 0
count = 0


# for session_0_center
mask = getMask(mask_path)


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    thresh = 1
    frameNo += 1

    if count%thresh==0:
        # Our operations on the frame come here
        img = bounding_pic(net, frame, scale, (416,416),mask)
        #510,496 1200,471 1527,616 706,749
        img = cv2.line(img, (510,496), (1200,471), (255,0,0), 3)
        img = cv2.line(img, (1200,471), (1527,616), (255,0,0), 3)
        img = cv2.line(img, (1527,616), (706,749), (255,0,0), 3)
        img = cv2.line(img, (706,749), (510,496), (255,0,0), 3)
        cv2.namedWindow('image',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('image', 800,600)
        cv2.imshow('image',img)
        outputVideo.write(frame)
    
    count += 1
    if count == thresh*30:
        count = 0
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
outputVideo.release()
cv2.destroyAllWindows()

# Find the position of vehicle

In [26]:
import numpy as np
def distanceFromPoint2Line(p, line):
    a,b,c = line[0],line[1],line[2]
    try:
        # if list or np.array shape 1
        x,y = p[0],p[1]
    except:
        # if np.array shape 2
        x,y = p[0][0], p[0][1]
    finally:
        top = abs(a*x+b*y+c)
        bottom = np.linalg.norm(np.array([a,b]))
        return top/bottom

def cosineVetorPhase(v1,v2):
    dotProduct = np.dot(v1,v2)
    normV1 = np.linalg.norm(v1)
    normV2 = np.linalg.norm(v2)
    return dotProduct/(normV1*normV2)

def checkSameSideNormalVector(p, line):
    # p is numpy array with shape [1,3] with the final element is 1
    # line is something like ax+by+c = 0, line =[a,b,c]
    return (line[0]*p[0]+line[1]*p[1]+c) > 0


# TRACKING

In [3]:
from yolo import *
from tracking import *

In [4]:
video_path = 'Z:\\Vehicle speed measuring\\video_speed measure\\front.avi'
yolo = Yolo(score = 0.3, iou = 0.45)
a = detect_video(yolo, video_path)









YOLOv3-416/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[[2.02398132e+02 5.99315674e+02 2.33738159e+02 6.36699341e+02
  6.00000000e+00]
 [4.20787262e+02 1.86356409e+03 4.70846680e+02 1.91694641e+03
  5.00000000e+00]
 [1.44748825e+02 2.15471466e+02 1.67223983e+02 2.42189254e+02
  4.00000000e+00]
 [2.35829880e+02 3.71403839e+02 2.71858276e+02 4.25982727e+02
  3.00000000e+00]
 [3.27419922e+02 1.42178009e+02 4.04702698e+02 2.22172272e+02
  2.00000000e+00]
 [2.67850373e+02 3.87499726e+02 3.22003784e+02 4.56348297e+02
  1.00000000e+00]]
[[2.07107895e+02 5.47363159e+02 2.36786560e+02 5.88953797e+02
  7.00000000e+00]
 [2.02406578e+02 5.98334963e+02 2.32259875e+02 6.32682199e+02
  6.00000000e+00]
 [4.20770346e+02 1.86349029e+03 4.70837109e+02 1.91694758e+03
  5.00000000e+00]
 [1.45105485e+02 2.16397160e+02 1.67635161e+02 2.43443961e+02
  4.00000000e+00]
 [2.36851836e+02 3.71757219e+02 2.75803689e

  1.00000000e+00]]
[[6.77984746e+02 1.76175560e+03 8.57790754e+02 1.91490911e+03
  9.00000000e+00]
 [4.20968284e+02 1.86354151e+03 4.70649897e+02 1.91728239e+03
  5.00000000e+00]
 [1.44705666e+02 2.33148849e+02 1.72911654e+02 2.70474950e+02
  4.00000000e+00]
 [2.56233759e+02 3.98706508e+02 2.97548377e+02 4.67202207e+02
  3.00000000e+00]
 [3.27285538e+02 1.42332962e+02 4.04667194e+02 2.22042351e+02
  2.00000000e+00]
 [2.95746329e+02 4.16513436e+02 3.59097257e+02 4.97941135e+02
  1.00000000e+00]]
[[6.68397879e+02 1.73383847e+03 8.40780461e+02 1.90849436e+03
  9.00000000e+00]
 [4.20846144e+02 1.86333137e+03 4.70704156e+02 1.91736747e+03
  5.00000000e+00]
 [1.44680353e+02 2.33449070e+02 1.73115920e+02 2.71849054e+02
  4.00000000e+00]
 [2.56252085e+02 4.01460138e+02 2.97674038e+02 4.68558597e+02
  3.00000000e+00]
 [3.27228474e+02 1.42335565e+02 4.04643021e+02 2.22009716e+02
  2.00000000e+00]
 [2.98127341e+02 4.19171300e+02 3.61263517e+02 5.00452621e+02
  1.00000000e+00]]
[[6.56568085e+02 1.

[[6.84367806e+02 1.57078906e+03 8.24870582e+02 1.78349971e+03
  1.10000000e+01]
 [5.19615605e+02 1.32446822e+03 6.17819404e+02 1.49297383e+03
  9.00000000e+00]
 [4.20333657e+02 1.86190916e+03 4.70968498e+02 1.91746846e+03
  5.00000000e+00]
 [1.52170636e+02 2.51365475e+02 1.79663230e+02 2.95101300e+02
  4.00000000e+00]
 [2.89733196e+02 4.48773770e+02 3.41183514e+02 5.28498025e+02
  3.00000000e+00]
 [3.27000772e+02 1.42531477e+02 4.04588010e+02 2.21701177e+02
  2.00000000e+00]
 [3.41572152e+02 4.66874581e+02 4.13917014e+02 5.56869515e+02
  1.00000000e+00]]
[[6.69789912e+02 1.53295436e+03 8.02537181e+02 1.73701205e+03
  1.10000000e+01]
 [5.09697122e+02 1.30523677e+03 6.05928676e+02 1.47031519e+03
  9.00000000e+00]
 [4.20391430e+02 1.86204803e+03 4.71142487e+02 1.91751719e+03
  5.00000000e+00]
 [1.52223640e+02 2.53060002e+02 1.79895798e+02 2.96749799e+02
  4.00000000e+00]
 [2.96440399e+02 4.56134443e+02 3.45824822e+02 5.34756127e+02
  3.00000000e+00]
 [3.27020118e+02 1.42555624e+02 4.04598

[[1.52189136e+02 2.86081714e+02 1.80476877e+02 3.23948371e+02
  1.20000000e+01]
 [5.11882172e+02 1.18206479e+03 6.11574400e+02 1.32753037e+03
  1.10000000e+01]
 [4.26369242e+02 1.10757928e+03 4.98282044e+02 1.21749889e+03
  9.00000000e+00]
 [4.20642650e+02 1.86224196e+03 4.71347471e+02 1.91715540e+03
  5.00000000e+00]
 [1.59806247e+02 2.62145011e+02 1.87923230e+02 3.10970460e+02
  4.00000000e+00]
 [3.38505794e+02 5.09446701e+02 3.98564065e+02 6.09576746e+02
  3.00000000e+00]
 [3.26832037e+02 1.42658032e+02 4.04726984e+02 2.21609344e+02
  2.00000000e+00]
 [4.08311644e+02 5.30558468e+02 4.92075859e+02 6.28937799e+02
  1.00000000e+00]]
[[1.57653481e+02 2.87094623e+02 1.84645038e+02 3.23307639e+02
  1.20000000e+01]
 [5.05804136e+02 1.16303949e+03 6.03800068e+02 1.30379202e+03
  1.10000000e+01]
 [4.22385324e+02 1.09487454e+03 4.93111861e+02 1.20047283e+03
  9.00000000e+00]
 [4.20688694e+02 1.86224475e+03 4.71255682e+02 1.91717158e+03
  5.00000000e+00]
 [1.60314985e+02 2.62049162e+02 1.88222

[[1.62457965e+02 3.24681157e+02 1.88883501e+02 3.61848484e+02
  1.40000000e+01]
 [4.24460261e+02 9.72046873e+02 4.89717692e+02 1.06602432e+03
  1.10000000e+01]
 [3.63925611e+02 9.43633879e+02 4.14790337e+02 1.03901546e+03
  9.00000000e+00]
 [4.20467564e+02 1.85345955e+03 4.70600089e+02 1.92022010e+03
  5.00000000e+00]
 [1.72749158e+02 2.69355036e+02 1.98728975e+02 3.09819192e+02
  4.00000000e+00]
 [3.83941011e+02 5.99092705e+02 4.68912052e+02 7.03286906e+02
  3.00000000e+00]
 [3.27063504e+02 1.42349242e+02 4.04793447e+02 2.22053094e+02
  2.00000000e+00]
 [4.90658869e+02 6.17096613e+02 6.03216961e+02 7.42212202e+02
  1.00000000e+00]]
[[1.62576837e+02 3.25329520e+02 1.89732257e+02 3.63791379e+02
  1.40000000e+01]
 [4.19644579e+02 9.59018463e+02 4.84417148e+02 1.05069199e+03
  1.10000000e+01]
 [3.61746982e+02 9.36492596e+02 4.11321196e+02 1.03116116e+03
  9.00000000e+00]
 [4.20742767e+02 1.85745411e+03 4.71097935e+02 1.92046385e+03
  5.00000000e+00]
 [1.73556808e+02 2.70271612e+02 1.99514

...

[[ 579.49969985 1399.21767955  726.24749925 1671.80539155   31.        ]
 [ 321.2048442   861.07254905  374.24258105  932.90556113   21.        ]
 [ 271.14138132  737.26381381  317.56764112  813.1630714    16.        ]
 [ 300.52442035  550.46746824  371.90694471  632.46476733   14.        ]
 [ 419.47664703 1848.87827162  470.06042228 1916.87501963    5.        ]
 [ 347.67884489  454.25760784  409.06930238  541.52474517    4.        ]
 [ 326.72001289  142.36041159  403.86609087  222.01147233    2.        ]]
[[ 565.26303931 1421.67960061  703.55742703 1668.74275995   31.        ]
 [ 318.74652373  855.28187876  372.01100589  927.85431762   21.        ]
 [ 269.06029194  733.33681371  315.00883212  804.29729775   16.        ]
 [ 306.85187539  552.78270211  379.42970507  638.91731729   14.        ]
 [ 419.30365307 1844.27997739  469.49620669 1915.89032446    5.        ]
 [ 351.30235035  458.86113941  413.86740986  546.10650365    4.        ]
 [ 326.66243998  142.38615991  403.92482438  221.9

[[ 484.55354598 1116.7560373   574.65555722 1258.46173274   35.        ]
 [ 485.2898931  1271.32071072  575.83486356 1395.85993547   31.        ]
 [ 283.79788194  786.88092583  335.47147849  844.83329216   21.        ]
 [ 249.52401991  698.45622662  293.27556637  745.13659623   16.        ]
 [ 347.20403373  616.41105805  431.31160986  715.77052131   14.        ]
 [ 420.63346379 1837.90078625  468.56447491 1919.57039902    5.        ]
 [ 408.68181255  520.5146752   487.79817307  622.08868865    4.        ]
 [ 326.79909529  142.27996008  404.30450907  222.04583882    2.        ]]
[[ 478.85657655 1100.67739181  566.16281825 1241.31723908   35.        ]
 [ 479.21927824 1253.57674951  567.38758717 1380.44079281   31.        ]
 [ 281.94754584  784.98235267  333.59233156  843.65354234   21.        ]
 [ 251.81183986  695.88365673  292.9432178   740.65122323   16.        ]
 [ 352.25195563  621.79849847  437.69265272  721.70936617   14.        ]
 [ 420.50500464 1837.90186951  468.59793577 1919.7

[[ 229.47444061  644.00757485  266.93660392  696.77977704   37.        ]
 [ 389.06354634  915.55486452  460.82411642 1010.6894342    35.        ]
 [ 407.23384051 1074.43227094  480.04178698 1170.08573343   31.        ]
 [ 259.89973363  718.3982345   298.3806756   767.8448349    21.        ]
 [ 426.33055763  740.59539363  537.55389149  856.73791409   14.        ]
 [ 420.28697573 1837.22732717  468.25859748 1919.97466611    5.        ]
 [ 532.9585816   647.80745708  645.95531209  782.68092083    4.        ]
 [ 327.24064437  141.94664641  404.85172467  222.17282124    2.        ]]
[[ 220.67833194  635.53382009  260.78219754  689.8456962    37.        ]
 [ 385.31306004  905.41996888  456.33268961 1001.44662697   35.        ]
 [ 403.22523709 1065.43759796  475.94499705 1158.2741236    31.        ]
 [ 257.25338817  716.35193131  296.72374528  765.85015061   21.        ]
 [ 433.25836058  752.5804328   544.93318155  870.22598126   14.        ]
 [ 420.24372346 1837.22600523  468.20281291 1919.9

[[ 208.49742727  607.57266117  245.78266031  662.8770492    37.        ]
 [ 343.55550418  815.11129301  394.45554298  892.32401978   35.        ]
 [ 358.18434158  958.97452052  421.190107   1045.72431142   31.        ]
 [ 519.30815852  906.49803646  678.67380861 1053.51015362   14.        ]
 [ 420.22152411 1837.34213281  468.51228859 1920.26095562    5.        ]
 [ 718.30409771  838.61188582  886.75898443 1028.47853621    4.        ]
 [ 335.17289087  139.4974285   405.25396381  223.05284357    2.        ]]
[[ 207.62903823  607.10410086  244.26359287  661.29268534   37.        ]
 [ 342.09413856  813.26414534  390.83901863  886.40148212   35.        ]
 [ 355.85519325  957.58522195  418.01546088 1040.33575306   31.        ]
 [ 534.80193157  926.38089007  697.37699546 1079.75142055   14.        ]
 [ 420.24430286 1837.44457364  468.57227801 1920.23133987    5.        ]
 [ 732.59092527  857.43747195  908.34213885 1057.34549079    4.        ]
 [ 335.21059107  139.43358781  405.14379543  223.1

KeyboardInterrupt: 

In [7]:
import numpy as np
a = [[1,2]]
a.append([2,3])
# print(np.lialg.norm(a[]))
print(a[-2])

[1, 2]
